In [1]:
from keras.layers import LSTM, Dense
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
song_df = pd.read_csv("data/songlyrics/songdata.csv")
song_df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
song_df.at[0, "text"]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?  \n  \nAnd when we go for a walk in the park  \nAnd she holds me and squeezes my hand  \nWe'll go on walking for hours and talking  \nAbout all the things that we plan  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?\n\n"

In [4]:
#Changed my mind
import re
punct = list(".,?!")

def clean_lyrics(lyrics):
    for p in punct:
        lyrics = lyrics.replace(p, " {} ".format(p))
    for p in ["(",")"]:
        lyrics = lyrics.replace(p, "")
    lyrics = re.sub("\s+", " ", lyrics)
    return np.array(["START"] + lyrics.lower().split() + ["END"])

In [5]:
def clean_lyrics(lyrics):
    lyrics = lyrics.replace("\n", "").lower()
    lyrics = re.sub("\s+", " ", lyrics)
    return "S" + lyrics + "E"

In [6]:
clean_lyrics(song_df.at[0, "text"])

"Slook at her face, it's a wonderful face and it means something special to me look at the way that she smiles when she sees me how lucky can one fellow be? she's just my kind of girl, she makes me feel fine who could ever believe that she could be mine? she's just my kind of girl, without her i'm blue and if she ever leaves me what could i do, what could i do? and when we go for a walk in the park and she holds me and squeezes my hand we'll go on walking for hours and talking about all the things that we plan she's just my kind of girl, she makes me feel fine who could ever believe that she could be mine? she's just my kind of girl, without her i'm blue and if she ever leaves me what could i do, what could i do?E"

In [7]:
song_df["clean"] = song_df.text.apply(clean_lyrics)

In [8]:
song_df.head()

,artist,song,link,text,clean
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","Slook at her face, it's a wonderful face and i..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","Stake it easy with me, please touch me gently ..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,Si'll never know why i had to go why i had to ...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,Smaking somebody happy is a question of give a...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,Smaking somebody happy is a question of give a...


In [9]:
data = song_df.clean.values
data[0]

"Slook at her face, it's a wonderful face and it means something special to me look at the way that she smiles when she sees me how lucky can one fellow be? she's just my kind of girl, she makes me feel fine who could ever believe that she could be mine? she's just my kind of girl, without her i'm blue and if she ever leaves me what could i do, what could i do? and when we go for a walk in the park and she holds me and squeezes my hand we'll go on walking for hours and talking about all the things that we plan she's just my kind of girl, she makes me feel fine who could ever believe that she could be mine? she's just my kind of girl, without her i'm blue and if she ever leaves me what could i do, what could i do?E"

In [10]:
from itertools import chain
word_set = set(chain(*data))
len(word_set)

51

In [11]:
len(list(chain(*data)))

63190685

In [12]:
N = len(data)
K = len(word_set)
SEQ_LEN = 64

In [13]:
letter2idx = dict((c, i) for i, c in enumerate(word_set))
idx2letter = dict((i, c) for i, c in enumerate(word_set))

In [20]:
def create_batch(data, n=128, p_start = .1):
    #Create a batch of n samples, each row in X representing SEQ_LEN letters from a song
    #with each row in y representing the one-hot encoding of the next letter (or the STOP character "S")
    #p_start determines the probability of starting at the beginning of the song vice a random point
    X = np.zeros((n, SEQ_LEN, K))
    y = np.zeros((n, K))
    
    for i in range(n):
        song_idx = np.random.choice(N)
        song_len = len(data[song_idx])
        if np.random.rand() < p_start:
            start_idx = 0
        else:
            start_idx = min(np.random.choice(song_len), song_len - SEQ_LEN - 1)
        
        for j, letter in enumerate(data[song_idx][start_idx:start_idx + SEQ_LEN]):
            letter_idx = letter2idx[letter]
            X[i, j, letter_idx] = 1
            
        next_letter_idx = letter2idx[data[song_idx][start_idx + SEQ_LEN]]
        y[i, next_letter_idx] = 1
    
    return X, y

In [22]:
X, y = create_batch(data)

In [32]:
blah = iter(range(len(X)))

In [67]:
i = next(blah)
"".join([idx2letter[idx] for idx in X[i].argmax(axis = 1)]), idx2letter[y[i].argmax()]

('e, not me one of us is winterhearted one of us is cold as ice on', 'e')

In [28]:
[idx2letter[idx] for idx in y.argmax(axis = 1)]

['t',
 ' ',
 't',
 't',
 'E',
 ' ',
 's',
 'i',
 'c',
 'y',
 'l',
 ' ',
 ' ',
 'u',
 'o',
 ' ',
 'g',
 ',',
 'i',
 'a',
 'E',
 ' ',
 'a',
 'a',
 'e',
 'r',
 ' ',
 'n',
 'e',
 's',
 't',
 ' ',
 "'",
 'd',
 'e',
 'E',
 'd',
 'n',
 'a',
 'e',
 'w',
 'f',
 ' ',
 'c',
 't',
 ' ',
 'h',
 ' ',
 'n',
 'a',
 ',',
 ' ',
 'd',
 'k',
 '(',
 'a',
 ' ',
 'b',
 ' ',
 'l',
 'o',
 'g',
 ' ',
 'v',
 ' ',
 'l',
 "'",
 'r',
 'd',
 'a',
 'e',
 'o',
 't',
 'e',
 'h',
 'l',
 'e',
 'i',
 'l',
 'r',
 'l',
 'g',
 'm',
 'w',
 'u',
 ' ',
 'j',
 ' ',
 'u',
 'h',
 'i',
 'r',
 'e',
 'a',
 'w',
 ' ',
 'r',
 ' ',
 'm',
 ' ',
 'f',
 's',
 'i',
 ' ',
 ' ',
 'm',
 'y',
 ' ',
 'a',
 'h',
 'E',
 'w',
 ' ',
 ' ',
 'g',
 'E',
 ' ',
 'd',
 'E',
 'e',
 'l',
 'a',
 'E',
 ' ',
 'o',
 'e',
 't',
 'c']

In [26]:
idx2letter

{0: "'",
 1: 'x',
 2: 'i',
 3: 'q',
 4: '5',
 5: '(',
 6: 'E',
 7: '"',
 8: 'k',
 9: 'r',
 10: 'v',
 11: ' ',
 12: '.',
 13: 't',
 14: 'z',
 15: '9',
 16: '4',
 17: '3',
 18: 'o',
 19: '0',
 20: 'j',
 21: ')',
 22: 'w',
 23: 'b',
 24: 'g',
 25: '1',
 26: '7',
 27: ',',
 28: 'd',
 29: '8',
 30: 'y',
 31: 'S',
 32: 'n',
 33: '2',
 34: 's',
 35: '[',
 36: ':',
 37: '!',
 38: 'l',
 39: 'a',
 40: 'e',
 41: '?',
 42: '6',
 43: 'u',
 44: 'h',
 45: '-',
 46: 'c',
 47: 'f',
 48: 'p',
 49: ']',
 50: 'm'}

In [27]:
letter2idx

{' ': 11,
 '!': 37,
 '"': 7,
 "'": 0,
 '(': 5,
 ')': 21,
 ',': 27,
 '-': 45,
 '.': 12,
 '0': 19,
 '1': 25,
 '2': 33,
 '3': 17,
 '4': 16,
 '5': 4,
 '6': 42,
 '7': 26,
 '8': 29,
 '9': 15,
 ':': 36,
 '?': 41,
 'E': 6,
 'S': 31,
 '[': 35,
 ']': 49,
 'a': 39,
 'b': 23,
 'c': 46,
 'd': 28,
 'e': 40,
 'f': 47,
 'g': 24,
 'h': 44,
 'i': 2,
 'j': 20,
 'k': 8,
 'l': 38,
 'm': 50,
 'n': 32,
 'o': 18,
 'p': 48,
 'q': 3,
 'r': 9,
 's': 34,
 't': 13,
 'u': 43,
 'v': 10,
 'w': 22,
 'x': 1,
 'y': 30,
 'z': 14}